In [1]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import os, gc
import pickle
import datetime, time
from tqdm import tqdm_notebook as tqdm


from sklearn import preprocessing

%matplotlib inline

In [2]:
path = '/Users/kanoumotoharu/Downloads/m5-forecasting-accuracy/'

In [3]:
os.listdir(path)

['calendar.csv',
 'sell_prices.csv',
 'sales_train_validation.csv',
 'sample_submission.csv']

In [4]:
%%time
train_df = pd.read_csv(path+'sales_train_validation.csv')
calendar_df = pd.read_csv(path+'calendar.csv')
sell_prices_df = pd.read_csv(path+'sell_prices.csv')
sample_submission_df = pd.read_csv(path+'sample_submission.csv')

CPU times: user 12.5 s, sys: 1.38 s, total: 13.9 s
Wall time: 14 s


In [5]:
train_df.head()

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,3,0,1,1,1,3,0,1,1
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,1,2,1,1,1,0,1,1,1
3,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,0,5,4,1,0,1,3,7,2
4,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,1,1,0,1,1,2,2,2,4


In [6]:
train_df.store_id.nunique()

10

In [7]:
calendar_df.head()

,date,wm_yr_wk,weekday,wday,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI
0,2011-01-29,11101,Saturday,1,1,2011,d_1,NaN,NaN,NaN,NaN,0,0,0
1,2011-01-30,11101,Sunday,2,1,2011,d_2,NaN,NaN,NaN,NaN,0,0,0
2,2011-01-31,11101,Monday,3,1,2011,d_3,NaN,NaN,NaN,NaN,0,0,0
3,2011-02-01,11101,Tuesday,4,2,2011,d_4,NaN,NaN,NaN,NaN,1,1,0
4,2011-02-02,11101,Wednesday,5,2,2011,d_5,NaN,NaN,NaN,NaN,1,0,1


In [8]:
sell_prices_df.head()

,store_id,item_id,wm_yr_wk,sell_price
0,CA_1,HOBBIES_1_001,11325,9.58
1,CA_1,HOBBIES_1_001,11326,9.58
2,CA_1,HOBBIES_1_001,11327,8.26
3,CA_1,HOBBIES_1_001,11328,8.26
4,CA_1,HOBBIES_1_001,11329,8.26


In [9]:
d_cols = [f'd_{i}' for i in range(1,1914)]

In [10]:
data = train_df.set_index('id')[d_cols].T
data.reset_index(drop=True, inplace=True)

In [11]:
calendar_df.date = pd.to_datetime(calendar_df.date)

In [12]:
calendar_df2 = calendar_df[:1913]

In [13]:
data = pd.concat([
    calendar_df2[['date', 'weekday']],
    data
],axis=1)

In [71]:
del calendar_df2

In [14]:
train_df.store_id.unique()

array(['CA_1', 'CA_2', 'CA_3', 'CA_4', 'TX_1', 'TX_2', 'TX_3', 'WI_1',
       'WI_2', 'WI_3'], dtype=object)

In [15]:
corr_dict = {}
store_id = ['CA_1', 'CA_2', 'CA_3', 'CA_4', 'TX_1', 'TX_2', 'TX_3', 'WI_1', 'WI_2', 'WI_3']
for store in tqdm(store_id):
    tmp_df = data.iloc[-84:,data.columns.str.contains(store)]
    mean = tmp_df.mean(axis=0)
    tmp_df = tmp_df.loc[:,mean>1.5]
    corr_dict[store] = tmp_df.iloc[-84:,tmp_df.columns.str.contains(store)].corr()

In [16]:
with open('corr_dict.pkl', mode='wb') as f:
    pickle.dump(corr_dict, f)

In [17]:
"""
for key, value in corr_dict.items():
    plt.figure(figsize=(30,30))
    sns.heatmap(value, cmap='coolwarm', annot=True)
    plt.show()
"""

"\nfor key, value in corr_dict.items():\n    plt.figure(figsize=(30,30))\n    sns.heatmap(value, cmap='coolwarm', annot=True)\n    plt.show()\n"

In [18]:
corr_dict.keys()

dict_keys(['CA_1', 'CA_2', 'CA_3', 'CA_4', 'TX_1', 'TX_2', 'TX_3', 'WI_1', 'WI_2', 'WI_3'])

In [19]:
corr = corr_dict['CA_1']
a = corr[abs(corr)>0.5].isnull().mean(axis=1)
a = a.sort_values()

In [20]:
a.head()

FOODS_3_540_CA_1_validation    0.967192
FOODS_3_147_CA_1_validation    0.971129
FOODS_3_592_CA_1_validation    0.972441
FOODS_3_714_CA_1_validation    0.972441
FOODS_3_618_CA_1_validation    0.973753
dtype: float64

In [21]:
#del train_df, calendar_df
gc.collect()

28

In [22]:
data2 = data.iloc[:,2:].rolling(70, min_periods=1).sum()
data3 = data.iloc[::-1,2:].rolling(70, min_periods=1).sum()

In [23]:
data3 = data3.iloc[::-1, :]

In [24]:
data2[data2==0]=np.nan
data3[data3==0]=np.nan

In [25]:
data2 = (data2+data3)/2

In [26]:
del data3
gc.collect()

In [27]:
data2 = pd.concat([
    data.iloc[:,:2],
    data2
], axis=1)

In [28]:
data3 = data.copy()
data3[data2.isnull()]=np.nan
data2 = data3
del data3

In [29]:
data.head()

,date,weekday,HOBBIES_1_001_CA_1_validation,HOBBIES_1_002_CA_1_validation,HOBBIES_1_003_CA_1_validation,HOBBIES_1_004_CA_1_validation,HOBBIES_1_005_CA_1_validation,HOBBIES_1_006_CA_1_validation,HOBBIES_1_007_CA_1_validation,HOBBIES_1_008_CA_1_validation,...,FOODS_3_818_WI_3_validation,FOODS_3_819_WI_3_validation,FOODS_3_820_WI_3_validation,FOODS_3_821_WI_3_validation,FOODS_3_822_WI_3_validation,FOODS_3_823_WI_3_validation,FOODS_3_824_WI_3_validation,FOODS_3_825_WI_3_validation,FOODS_3_826_WI_3_validation,FOODS_3_827_WI_3_validation
0,2011-01-29,Saturday,0,0,0,0,0,0,0,12,...,0,14,1,0,4,0,0,0,0,0
1,2011-01-30,Sunday,0,0,0,0,0,0,0,15,...,0,11,1,0,4,0,0,6,0,0
2,2011-01-31,Monday,0,0,0,0,0,0,0,0,...,0,5,1,0,2,2,0,0,0,0
3,2011-02-01,Tuesday,0,0,0,0,0,0,0,0,...,0,6,1,0,5,2,0,2,0,0
4,2011-02-02,Wednesday,0,0,0,0,0,0,0,0,...,0,5,1,0,2,0,0,2,0,0


In [30]:
data2.head()

,date,weekday,HOBBIES_1_001_CA_1_validation,HOBBIES_1_002_CA_1_validation,HOBBIES_1_003_CA_1_validation,HOBBIES_1_004_CA_1_validation,HOBBIES_1_005_CA_1_validation,HOBBIES_1_006_CA_1_validation,HOBBIES_1_007_CA_1_validation,HOBBIES_1_008_CA_1_validation,...,FOODS_3_818_WI_3_validation,FOODS_3_819_WI_3_validation,FOODS_3_820_WI_3_validation,FOODS_3_821_WI_3_validation,FOODS_3_822_WI_3_validation,FOODS_3_823_WI_3_validation,FOODS_3_824_WI_3_validation,FOODS_3_825_WI_3_validation,FOODS_3_826_WI_3_validation,FOODS_3_827_WI_3_validation
0,2011-01-29,Saturday,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.0,...,NaN,14.0,1,NaN,4.0,NaN,NaN,NaN,NaN,NaN
1,2011-01-30,Sunday,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15.0,...,NaN,11.0,1,NaN,4.0,NaN,NaN,6.0,NaN,NaN
2,2011-01-31,Monday,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,...,NaN,5.0,1,NaN,2.0,2.0,NaN,0.0,NaN,NaN
3,2011-02-01,Tuesday,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,...,NaN,6.0,1,NaN,5.0,2.0,NaN,2.0,NaN,NaN
4,2011-02-02,Wednesday,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,...,NaN,5.0,1,NaN,2.0,0.0,NaN,2.0,NaN,NaN


In [32]:
product_cols = data.columns[2:]

In [45]:
new_df = pd.DataFrame()
for window in [7,21, 35]: 
    tmpdata = data[product_cols].shift(1).rolling(window, min_periods=1).mean()
    tmpdata.columns = [f'rolling{window}_{col}' for col in tmpdata.columns]
    new_df = pd.concat([
        new_df,
        tmpdata
    ], axis=1)

In [46]:
new_df.head()

,rolling7_HOBBIES_1_001_CA_1_validation,rolling7_HOBBIES_1_002_CA_1_validation,rolling7_HOBBIES_1_003_CA_1_validation,rolling7_HOBBIES_1_004_CA_1_validation,rolling7_HOBBIES_1_005_CA_1_validation,rolling7_HOBBIES_1_006_CA_1_validation,rolling7_HOBBIES_1_007_CA_1_validation,rolling7_HOBBIES_1_008_CA_1_validation,rolling7_HOBBIES_1_009_CA_1_validation,rolling7_HOBBIES_1_010_CA_1_validation,...,rolling35_FOODS_3_818_WI_3_validation,rolling35_FOODS_3_819_WI_3_validation,rolling35_FOODS_3_820_WI_3_validation,rolling35_FOODS_3_821_WI_3_validation,rolling35_FOODS_3_822_WI_3_validation,rolling35_FOODS_3_823_WI_3_validation,rolling35_FOODS_3_824_WI_3_validation,rolling35_FOODS_3_825_WI_3_validation,rolling35_FOODS_3_826_WI_3_validation,rolling35_FOODS_3_827_WI_3_validation
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12.00,2.0,0.000000,...,0.0,14.0,1.0,0.0,4.000000,0.000000,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,13.50,1.0,0.000000,...,0.0,12.5,1.0,0.0,4.000000,0.000000,0.0,3.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.00,3.0,0.333333,...,0.0,10.0,1.0,0.0,3.333333,0.666667,0.0,2.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.75,3.0,0.250000,...,0.0,9.0,1.0,0.0,3.750000,1.000000,0.0,2.0,0.0,0.0


In [63]:
sell_prices_df.head()

,store_id,item_id,wm_yr_wk,sell_price
0,CA_1,HOBBIES_1_001,11325,9.58
1,CA_1,HOBBIES_1_001,11326,9.58
2,CA_1,HOBBIES_1_001,11327,8.26
3,CA_1,HOBBIES_1_001,11328,8.26
4,CA_1,HOBBIES_1_001,11329,8.26


In [72]:
sell_prices_df['date'] = sell_prices_df['wm_yr_wk'].map(calendar_df.set_index('wm_yr_wk')['date'].to_dict())

In [94]:
sell_prices_df.head()

,store_id,item_id,wm_yr_wk,sell_price,date
0,CA_1,HOBBIES_1_001,11325,9.58,2013-07-19
1,CA_1,HOBBIES_1_001,11326,9.58,2013-07-26
2,CA_1,HOBBIES_1_001,11327,8.26,2013-08-02
3,CA_1,HOBBIES_1_001,11328,8.26,2013-08-09
4,CA_1,HOBBIES_1_001,11329,8.26,2013-08-16


In [102]:
sell_prices_df.sort_values('date', inplace=True)
a = sell_prices_df.groupby(['store_id', 'item_id'])#['sell_price']
#a = a.apply(lambda x: x.sort_values('date'))

In [104]:
a.sell_price.mean()

store_id  item_id        
CA_1      FOODS_1_001         2.169362
          FOODS_1_002         8.937447
          FOODS_1_003         2.976809
          FOODS_1_004         1.851378
          FOODS_1_005         3.334468
          FOODS_1_006         2.223085
          FOODS_1_008         3.382939
          FOODS_1_009         2.397511
          FOODS_1_010         5.374460
          FOODS_1_011         2.520957
          FOODS_1_012         5.502773
          FOODS_1_013         1.471596
          FOODS_1_014         0.980000
          FOODS_1_015         3.207071
          FOODS_1_016         2.074113
          FOODS_1_017         2.360446
          FOODS_1_018         0.981348
          FOODS_1_019         1.047411
          FOODS_1_020         2.655106
          FOODS_1_021         0.980000
          FOODS_1_022         4.607115
          FOODS_1_023         5.501016
          FOODS_1_024         4.705426
          FOODS_1_025         4.109412
          FOODS_1_026         6.954265